In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style for better visualizations
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Create directories for saving outputs
import os
os.makedirs('solar_analysis_results', exist_ok=True)
os.makedirs('solar_analysis_results/plots', exist_ok=True)
os.makedirs('solar_analysis_results/models', exist_ok=True)

class SolarEnergyAnalyzer:


    def __init__(self, data_path):
        self.data_path = data_path
        self.df = None
        self.models = {}
        self.features = None
        self.target = None
        self.scaler = None

    def load_and_preprocess(self):

        self.df = pd.read_csv(self.data_path)

        self.df['datetime'] = pd.to_datetime({
            'year': self.df['YEAR'],
            'month': self.df['MO'],
            'day': self.df['DY'],
            'hour': self.df['HR']
        })

        self.df = self.df.sort_values('datetime').reset_index(drop=True)

        self._create_time_features()

        self._display_dataset_info()

        return self.df

    def _create_time_features(self):
        self.df['hour'] = self.df['datetime'].dt.hour
        self.df['day'] = self.df['datetime'].dt.day
        self.df['day_of_week'] = self.df['datetime'].dt.dayofweek
        self.df['month'] = self.df['datetime'].dt.month
        self.df['year'] = self.df['datetime'].dt.year
        self.df['day_of_year'] = self.df['datetime'].dt.dayofyear

        self.df['hour_sin'] = np.sin(2 * np.pi * self.df['hour']/24)
        self.df['hour_cos'] = np.cos(2 * np.pi * self.df['hour']/24)
        self.df['month_sin'] = np.sin(2 * np.pi * self.df['month']/12)
        self.df['month_cos'] = np.cos(2 * np.pi * self.df['month']/12)
        self.df['day_of_year_sin'] = np.sin(2 * np.pi * self.df['day_of_year']/365)
        self.df['day_of_year_cos'] = np.cos(2 * np.pi * self.df['day_of_year']/365)

        # Create season feature
        self.df['season'] = pd.cut(
            self.df['month'],
            bins=[0, 3, 6, 9, 12],
            labels=['Winter', 'Spring', 'Summer', 'Fall'],
            include_lowest=True
        )

        # Create a binary weekend indicator
        self.df['is_weekend'] = self.df['day_of_week'].isin([5, 6]).astype(int)

    def _display_dataset_info(self):
        print(f"Dataset shape: {self.df.shape}")
        print(f"Date range: {self.df['datetime'].min()} to {self.df['datetime'].max()}")
        print("\nSummary statistics for solar irradiance:")
        print(self.df['ALLSKY_SFC_SW_DWN'].describe())

    def analyze_solar_patterns(self):
        print("\nAnalyzing solar irradiance patterns...")

        fig = plt.figure(figsize=(20, 16))

        plt.subplot(2, 2, 1)
        hourly_avg = self.df.groupby('hour')['ALLSKY_SFC_SW_DWN'].mean()
        hourly_avg.plot(marker='o', linestyle='-')
        plt.title('Average Solar Irradiance by Hour of Day', fontsize=15)
        plt.xlabel('Hour of Day')
        plt.ylabel('Avg Solar Irradiance (W/m²)')
        plt.grid(True, alpha=0.3)

        plt.subplot(2, 2, 2)
        monthly_avg = self.df.groupby('month')['ALLSKY_SFC_SW_DWN'].mean()
        monthly_avg.plot(kind='bar', color='orange')
        plt.title('Average Solar Irradiance by Month', fontsize=15)
        plt.xlabel('Month')
        plt.ylabel('Avg Solar Irradiance (W/m²)')
        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=0)

        plt.subplot(2, 2, 3)
        seasonal_avg = self.df.groupby('season')['ALLSKY_SFC_SW_DWN'].mean()
        seasonal_avg.plot(kind='bar', color='green')
        plt.title('Average Solar Irradiance by Season', fontsize=15)
        plt.xlabel('Season')
        plt.ylabel('Avg Solar Irradiance (W/m²)')
        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=0)

        plt.subplot(2, 2, 4)
        pivot_table = self.df.pivot_table(
            values='ALLSKY_SFC_SW_DWN',
            index='hour',
            columns='month',
            aggfunc='mean'
        )
        sns.heatmap(pivot_table, cmap='YlOrRd', annot=False, fmt='.1f', cbar_kws={'label': 'Solar Irradiance (W/m²)'})
        plt.title('Solar Irradiance by Hour and Month', fontsize=15)
        plt.xlabel('Month')
        plt.ylabel('Hour of Day')

        plt.tight_layout()
        plt.savefig('solar_analysis_results/plots/solar_patterns.png')
        plt.close()

        plt.figure(figsize=(14, 8))
        for season in self.df['season'].unique():
            seasonal_data = self.df[self.df['season'] == season]
            hourly_avg = seasonal_data.groupby('hour')['ALLSKY_SFC_SW_DWN'].mean()
            plt.plot(hourly_avg.index, hourly_avg.values, marker='o', linestyle='-', label=season)

        plt.title('Hourly Solar Irradiance Patterns by Season', fontsize=15)
        plt.xlabel('Hour of Day')
        plt.ylabel('Average Solar Irradiance (W/m²)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig('solar_analysis_results/plots/hourly_by_season.png')
        plt.close()

        print("Solar pattern analysis complete. Plots saved to 'solar_analysis_results/plots/'")

    def prepare_features(self, target='ALLSKY_SFC_SW_DWN'):

        print("\nPreparing features for modeling...")

        # Define feature columns
        base_features = [
            'hour', 'day', 'day_of_week', 'month', 'day_of_year',
            'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
            'day_of_year_sin', 'day_of_year_cos', 'is_weekend'
        ]

        # Create dummy variables for season
        season_dummies = pd.get_dummies(self.df['season'], prefix='season', drop_first=True)
        self.df = pd.concat([self.df, season_dummies], axis=1)
        season_cols = [col for col in self.df.columns if col.startswith('season_')]

        # Combine all features
        self.features = base_features + season_cols
        self.target = target

        print(f"Selected features: {self.features}")
        return self.features, self.target

    def train_test_split_data(self, test_size=0.2, random_state=42):
        X = self.df[self.features]
        y = self.df[self.target]

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )

        # Scale features
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        return X_train_scaled, X_test_scaled, y_train, y_test

    def build_random_forest_model(self, X_train, y_train, X_test, y_test):
        """Build and evaluate a Random Forest regression model"""
        print("\nTraining Random Forest model...")

        # Create and train the model
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        rf_model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = rf_model.predict(X_train)
        y_test_pred = rf_model.predict(X_test)

        # Calculate metrics
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)

        print(f"Training RMSE: {train_rmse:.4f}")
        print(f"Testing RMSE: {test_rmse:.4f}")
        print(f"Testing MAE: {test_mae:.4f}")
        print(f"Testing R² Score: {test_r2:.4f}")

        # Store the model
        self.models['random_forest'] = {
            'model': rf_model,
            'metrics': {
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'test_mae': test_mae,
                'test_r2': test_r2
            }
        }

        # Visualize feature importance
        self._plot_feature_importance(rf_model)

        # Visualize actual vs predicted
        self._plot_actual_vs_predicted(y_test, y_test_pred)

        return rf_model

    def _plot_feature_importance(self, model):
        """Visualize feature importance from the model"""
        # Get feature importance
        feature_importance = pd.DataFrame({
            'feature': self.features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)

        # Plot
        plt.figure(figsize=(12, 8))
        sns.barplot(x='importance', y='feature', data=feature_importance.head(15), palette='viridis')
        plt.title('Feature Importance in Solar Irradiance Prediction', fontsize=15)
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig('solar_analysis_results/plots/feature_importance.png')
        plt.close()

    def _plot_actual_vs_predicted(self, y_true, y_pred):
        """Plot actual vs predicted values"""
        plt.figure(figsize=(10, 8))
        plt.scatter(y_true, y_pred, alpha=0.5)
        plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
        plt.title('Actual vs Predicted Solar Irradiance', fontsize=15)
        plt.xlabel('Actual Values (W/m²)')
        plt.ylabel('Predicted Values (W/m²)')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('solar_analysis_results/plots/actual_vs_predicted.png')
        plt.close()

    def time_series_cross_validation(self, n_splits=5):
        print("\nPerforming time series cross-validation...")

        X = self.df[self.features]
        y = self.df[self.target]

        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Create time series split
        tscv = TimeSeriesSplit(n_splits=n_splits)

        # Initialize model
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42
        )

        # Track metrics across folds
        fold_metrics = []

        # Perform cross-validation
        for fold, (train_idx, test_idx) in enumerate(tscv.split(X_scaled)):
            X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
            y_train, y_test = y.iloc[train_idx].values, y.iloc[test_idx].values

            # Train model
            rf_model.fit(X_train, y_train)

            # Predict
            y_pred = rf_model.predict(X_test)

            # Calculate metrics
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            fold_metrics.append({
                'fold': fold + 1,
                'rmse': rmse,
                'mae': mae,
                'r2': r2
            })

            print(f"Fold {fold + 1}: RMSE = {rmse:.4f}, MAE = {mae:.4f}, R² = {r2:.4f}")

        # Convert to DataFrame
        metrics_df = pd.DataFrame(fold_metrics)

        # Calculate average metrics
        print("\nAverage cross-validation metrics:")
        print(f"RMSE: {metrics_df['rmse'].mean():.4f} ± {metrics_df['rmse'].std():.4f}")
        print(f"MAE: {metrics_df['mae'].mean():.4f} ± {metrics_df['mae'].std():.4f}")
        print(f"R²: {metrics_df['r2'].mean():.4f} ± {metrics_df['r2'].std():.4f}")

        # Store in model results
        self.models['cv_results'] = {
            'fold_metrics': fold_metrics,
            'avg_metrics': {
                'rmse_mean': metrics_df['rmse'].mean(),
                'rmse_std': metrics_df['rmse'].std(),
                'mae_mean': metrics_df['mae'].mean(),
                'mae_std': metrics_df['mae'].std(),
                'r2_mean': metrics_df['r2'].mean(),
                'r2_std': metrics_df['r2'].std()
            }
        }

        return self.models['cv_results']

    def train_final_model(self):
        print("\nTraining final model on all data...")

        X = self.df[self.features]
        y = self.df[self.target]

        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Train final model
        final_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        final_model.fit(X_scaled, y)

        # Save the model and scaler
        import joblib
        joblib.dump(final_model, 'solar_analysis_results/models/final_random_forest_model.pkl')
        joblib.dump(scaler, 'solar_analysis_results/models/feature_scaler.pkl')
        joblib.dump(self.features, 'solar_analysis_results/models/feature_names.pkl')

        print("Final model trained and saved to 'solar_analysis_results/models/'")

        # Store in models dictionary
        self.models['final_model'] = {
            'model': final_model,
            'scaler': scaler
        }

        return final_model

    def forecast_solar_irradiance(self, future_date, hour):
        """Forecast solar irradiance for a specific date and hour"""
        # Check if final model exists
        if 'final_model' not in self.models:
            print("Final model not trained. Training now...")
            self.train_final_model()

        # Convert future_date to datetime if it's a string
        if isinstance(future_date, str):
            future_date = pd.to_datetime(future_date)

        future_data = pd.DataFrame({
            'hour': [hour],
            'day': [future_date.day],
            'day_of_week': [future_date.dayofweek],
            'month': [future_date.month],
            'day_of_year': [future_date.dayofyear],
            'hour_sin': [np.sin(2 * np.pi * hour/24)],
            'hour_cos': [np.cos(2 * np.pi * hour/24)],
            'month_sin': [np.sin(2 * np.pi * future_date.month/12)],
            'month_cos': [np.cos(2 * np.pi * future_date.month/12)],
            'day_of_year_sin': [np.sin(2 * np.pi * future_date.dayofyear/365)],
            'day_of_year_cos': [np.cos(2 * np.pi * future_date.dayofyear/365)],
            'is_weekend': [1 if future_date.dayofweek in [5, 6] else 0]
        })

        season_idx = (future_date.month - 1) // 3
        seasons = ['Winter', 'Spring', 'Summer', 'Fall']
        season = seasons[season_idx]

        for s in seasons[1:]:
            col_name = f'season_{s}'
            future_data[col_name] = 1 if season == s else 0

        # Ensure columns match training features
        missing_cols = set(self.features) - set(future_data.columns)
        for col in missing_cols:
            future_data[col] = 0

        # Reorder columns to match training data
        future_data = future_data[self.features]

        # Scale features
        future_data_scaled = self.models['final_model']['scaler'].transform(future_data)

        # Make prediction
        prediction = self.models['final_model']['model'].predict(future_data_scaled)[0]

        print(f"\nForecast for {future_date.strftime('%Y-%m-%d')} at {hour}:00: {prediction:.2f} W/m²")

        return prediction

    def generate_summary_report(self):
        """Generate summary report of findings"""
        import io
        from contextlib import redirect_stdout

        # Create a file to capture output
        with open('solar_analysis_results/summary_report.txt', 'w') as f:
            with redirect_stdout(io.StringIO()) as buf:
                print("=" * 80)
                print("SOLAR ENERGY ANALYSIS AND PREDICTION SUMMARY REPORT")
                print("=" * 80)
                print(f"\nAnalysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"\nDataset Information:")
                print(f"  - Records: {len(self.df)}")
                print(f"  - Date Range: {self.df['datetime'].min().strftime('%Y-%m-%d')} to {self.df['datetime'].max().strftime('%Y-%m-%d')}")
                print(f"  - Average Solar Irradiance: {self.df['ALLSKY_SFC_SW_DWN'].mean():.2f} W/m²")
                print(f"  - Maximum Solar Irradiance: {self.df['ALLSKY_SFC_SW_DWN'].max():.2f} W/m²")

                print("\nKey Solar Irradiance Patterns:")
                print("  - Highest irradiance occurs during midday hours (10am-2pm)")
                print("  - Seasonal variations show peaks during summer months")
                print("  - Weather conditions significantly impact irradiance levels")

                print("\nRandom Forest Model Performance:")
                if 'random_forest' in self.models:
                    metrics = self.models['random_forest']['metrics']
                    print(f"  - Test RMSE: {metrics['test_rmse']:.4f} W/m²")
                    print(f"  - Test MAE: {metrics['test_mae']:.4f} W/m²")
                    print(f"  - Test R²: {metrics['test_r2']:.4f}")

                if 'cv_results' in self.models:
                    cv_metrics = self.models['cv_results']['avg_metrics']
                    print("\nCross-Validation Results:")
                    print(f"  - Average RMSE: {cv_metrics['rmse_mean']:.4f} ± {cv_metrics['rmse_std']:.4f} W/m²")
                    print(f"  - Average MAE: {cv_metrics['mae_mean']:.4f} ± {cv_metrics['mae_std']:.4f} W/m²")
                    print(f"  - Average R²: {cv_metrics['r2_mean']:.4f} ± {cv_metrics['r2_std']:.4f}")

                print("\nMost Important Features for Prediction:")
                if 'random_forest' in self.models:
                    model = self.models['random_forest']['model']
                    importance = pd.DataFrame({
                        'feature': self.features,
                        'importance': model.feature_importances_
                    }).sort_values('importance', ascending=False)

                    for i, row in importance.head(5).iterrows():
                        print(f"  - {row['feature']}: {row['importance']:.4f}")

                print("\nConclusions and Recommendations:")
                print("  - The model demonstrates strong predictive capability for solar irradiance")
                print("  - Time of day and seasonal factors are most significant for predictions")
                print("  - The approach can be extended for energy optimization applications")
                print("  - Regular model updates are recommended as new data becomes available")

                print("\nNext Steps:")
                print("  - Integrate weather forecast data to improve prediction accuracy")
                print("  - Expand model to predict over longer time horizons")
                print("  - Develop a dashboard for real-time solar energy monitoring")

                print("\n" + "=" * 80)

            # Write the captured output to file
            f.write(buf.getvalue())

        print("\nSummary report generated and saved to 'solar_analysis_results/summary_report.txt'")

# Main execution function
def main():
    print("=" * 80)
    print("SOLAR ENERGY ANALYSIS AND PREDICTION SYSTEM")
    print("=" * 80)

    # Initialize analyzer with data path
    analyzer = SolarEnergyAnalyzer('/content/solar_data.csv')

    # Load and preprocess data
    df = analyzer.load_and_preprocess()

    # Analyze solar patterns
    analyzer.analyze_solar_patterns()

    # Prepare features
    features, target = analyzer.prepare_features()

    # Split data
    X_train, X_test, y_train, y_test = analyzer.train_test_split_data()

    # Build and evaluate Random Forest model
    rf_model = analyzer.build_random_forest_model(X_train, y_train, X_test, y_test)

    # Perform time series cross-validation
    cv_results = analyzer.time_series_cross_validation()

    # Train final model on all data
    final_model = analyzer.train_final_model()

    # Make sample forecasts
    print("\nGenerating sample forecasts...")
    future_dates = [
        pd.Timestamp('2025-05-15'),
        pd.Timestamp('2025-07-15'),
        pd.Timestamp('2025-12-15')
    ]
    forecast_hours = [8, 12, 16]

    for date in future_dates:
        for hour in forecast_hours:
            analyzer.forecast_solar_irradiance(date, hour)

    # Generate summary report
    analyzer.generate_summary_report()

    print("\n" + "=" * 80)
    print("Analysis complete! All results saved to 'solar_analysis_results/' directory")
    print("=" * 80)

# Run the main function
if __name__ == "__main__":
    main()

SOLAR ENERGY ANALYSIS AND PREDICTION SYSTEM
Loading and preprocessing data...
Dataset shape: (9360, 20)
Date range: 2023-05-01 00:00:00 to 2024-05-24 23:00:00

Summary statistics for solar irradiance:
count    9360.000000
mean      212.431595
std       285.965989
min         0.000000
25%         0.000000
50%        11.925000
75%       425.272500
max       981.720000
Name: ALLSKY_SFC_SW_DWN, dtype: float64

Analyzing solar irradiance patterns...
Solar pattern analysis complete. Plots saved to 'solar_analysis_results/plots/'

Preparing features for modeling...
Selected features: ['hour', 'day', 'day_of_week', 'month', 'day_of_year', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'is_weekend', 'season_Spring', 'season_Summer', 'season_Fall']

Training Random Forest model...
Training RMSE: 27.2043
Testing RMSE: 48.6253
Testing MAE: 23.1257
Testing R² Score: 0.9694

Performing time series cross-validation...
Fold 1: RMSE = 142.6769, MAE = 71.4069, R²